# ISI MINT Model Catalog Queries

This notebook demonstrates how to construct and execute SPARQL queries using ISI's [MINT Model Catalog](http://ontosoft.isi.edu:8001/api/KnowledgeCaptureAndDiscovery/MINT-ModelCatalogQueries#/).

Requires [SPARQLWrapper](https://pypi.org/project/SPARQLWrapper/) which can be installed with:

```
pip install SPARQLWrapper
```

This enables a user to submit SPARQL queries to the current Model Catalog endpoint: [http://ontosoft.isi.edu:3030/ds/query](http://ontosoft.isi.edu:3030/ds/query). Full documentation on the available queries are described in the [Model Catalog API specs](http://ontosoft.isi.edu:8001/api/KnowledgeCaptureAndDiscovery/MINT-ModelCatalogQueries#/).

In [23]:
from SPARQLWrapper import SPARQLWrapper, JSON
import json

In [10]:
sparql = SPARQLWrapper("http://ontosoft.isi.edu:3030/ds/query")

### Get list of all model categories

In [72]:
sparql.setQuery("""
    PREFIX mc: <https://w3id.org/mint/modelCatalog#>

    SELECT distinct ?category
    WHERE {
      ?model a mc:Model; 
             mc:hasModelCategory ?category
    }
""")

In [73]:
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

Print the list of categories:

In [78]:
for i in results['results']['bindings']:
    print(i['category']['value'])

Hydrology
Land Use
Agriculture
Economic


### Get inputs/outputs referencing a model variable (standard name)

In [32]:
standard_name = 'atmosphere_water__precipitation_volume_flux'

Note that the `standard_name` variable is inserted into the SPARQL query at line 10 below:

In [39]:
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX mc: <https://w3id.org/mint/modelCatalog#>
    PREFIX os: <http://ontosoft.org/software#>

    SELECT DISTINCT ?io ?model_config ?modelVersion ?model
    WHERE {
      ?io a mc:DatasetSpecification.
      ?io mc:hasPresentation ?presentation.
      ?presentation mc:hasStandardVariable ?%s.
      ?model os:hasSoftwareVersion ?modelVersion.
      ?modelVersion mc:hasConfiguration ?model_config.
      {  
        ?model_config mc:hasInput ?io.
      }
      UNION
      {
        ?model_config mc:hasOutput ?io.
      }
    }
""" % standard_name)

In [40]:
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

Print the first result item:

In [70]:
results['results']['bindings'][0]

{'prop': {'type': 'uri',
  'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'},
 'value': {'type': 'uri',
  'value': 'https://w3id.org/mint/modelCatalog#VariablePresentation'}}

### Get resource (data) information

In [64]:
resource = 'TOPOFLOW'

Note that the `resource` variable is inserted into the SPARQL query at line 4 below:

In [65]:
sparql.setQuery("""
    SELECT ?prop ?value
    WHERE {
      ?%s ?prop ?value
    } 
""" % resource)

In [67]:
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

Print the first result item:

In [71]:
results['results']['bindings'][0]

{'prop': {'type': 'uri',
  'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'},
 'value': {'type': 'uri',
  'value': 'https://w3id.org/mint/modelCatalog#VariablePresentation'}}